In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import PIL.Image as Image


import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras as ks
from tensorflow.keras import datasets, layers, models

#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix,classification_report



In [2]:
IMAGE_SHAPE=(224,224)
classfier= tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4",input_shape=IMAGE_SHAPE+(3,))
])

In [3]:
IMAGE_SHAPE=(224,224)
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir=ks.utils.get_file("flower_photos", origin=dataset_url,cache_dir='.',untar=True)
import pathlib
data_dir= pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count
flowers_images_dict={
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}
flowers_labels_dict={
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}
x,y=[],[]
for flower_name, images in flowers_images_dict.items():
    for img_path in images:
        img=cv2.imread(str(img_path))
        resized_img=cv2.resize(img,IMAGE_SHAPE)
        x.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

x=np.array(x)
y=np.array(y)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0)
x_train_scaled=x_train/255
x_test_scaled=x_test/255

In [4]:


flower_classfier= tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",input_shape=IMAGE_SHAPE+(3,), trainable=False),
    layers.Dense(5)
])
flower_classfier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [5]:
flower_classfier.compile(optimizer='adam',
             loss=ks.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])



In [6]:
flower_classfier.fit(x_train_scaled,y_train,epochs=5)


Epoch 1/5


ResourceExhaustedError: in user code:

    File "C:\Users\Hussein ElBendry\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Hussein ElBendry\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Hussein ElBendry\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Hussein ElBendry\.conda\envs\tf\lib\site-packages\keras\engine\training.py", line 997, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\Hussein ElBendry\.conda\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 579, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "C:\Users\Hussein 